# Per-Sample Bottleneck 

This notebook shows how to apply the Per-Sample Bottleneck to pretrained ImageNet models. 

Ensure that `./data/imagenet` points to your copy of the ImageNet dataset. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torchvision
import torch
from torch import nn

import torchvision.models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize, Normalize
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt 
import os
from tqdm import tqdm_notebook
import json
import itertools
from PIL import Image
import numpy as np


from per_sample_bottleneck.per_sample_bottleneck import PerSampleBottleneck, insert_into_sequential
from per_sample_bottleneck.utils import get_output_shapes, plot_heatmap, denormalize

## Loading Data and Model

In [ ]:
imagenet_dir = 'data/imagenet/'

dev = torch.device('cuda:0')

# select a model to analyse
# model = torchvision.models.vgg16(pretrained=True)
model = torchvision.models.resnet50(pretrained=True)
# model = torchvision.models.inception_v3(pretrained=True)
model.eval()
model.to(dev)

# load the data
if type(model) == torchvision.models.inception.Inception3:
    image_size = 299
else:
    image_size = 224
    
valset = ImageFolder(
    os.path.join(imagenet_dir, 'validation'),
    transform=Compose([
        CenterCrop(256), Resize(image_size), ToTensor(), 
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

trainset = ImageFolder(
    os.path.join(imagenet_dir, 'train'),
    transform=Compose([
        CenterCrop(256), Resize(image_size), ToTensor(), 
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

with open('imagenet_class_index.json') as f:
    idx2class = {int(k): v[1] for k, v in json.load(f).items()}

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
img, target = valset[0]

### Insert the bottleneck into the model

You can experiment with the location of the bottleneck.

In [ ]:
sizes = get_output_shapes(model, img[None].to(dev))


if type(model) == torchvision.models.vgg.VGG:
    size = sizes['features.17']
    btln = PerSampleBottleneck(*size)
    model.features = insert_into_sequential(model.features, btln, 18)
elif type(model) == torchvision.models.resnet.ResNet:
    size = sizes['layer2']
    btln = PerSampleBottleneck(*size)
    model.layer2 = nn.Sequential(model.layer2, btln)
elif type(model) == torchvision.models.inception.Inception3:
    size = sizes['Mixed_5b']
    btln = PerSampleBottleneck(*size)
    model.Mixed_5b = nn.Sequential(model.Mixed_5b, btln)

if sum(type(m) == PerSampleBottleneck for m in model.modules()) > 1:
    raise ValueError("You added multiple bottlenecks to the model.")
    
model.to(dev)

## Estimate Mean and Variance

Here, we estimate the mean and variances of the feature map. It is important for measuring the amount of information transmitted.

In [ ]:
btln.estimate(model, trainloader, device=dev, n_samples=5000, progbar=True)

In [ ]:
neuron = (12, 3, 4)
print("Neuron at position {:} has mean {:.2f} and std {:.2f}".format(
    neuron, btln.estimator.mean()[neuron],  btln.estimator.std()[neuron]))

## Display Heatmaps for some random samples



In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(20, 6))
np.random.seed(0)
for ax, sample_idx in zip(axes.flatten(), np.random.choice(50000, 10)):
    img, target = valset[sample_idx]
    img = img[None].to(dev)
    
    # execute the model on a given sample and return the target NLL
    model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target].mean()
    
    # generate the heatmap
    heatmap = btln.heatmap(img, model_loss_closure)
    
    # plot the heatmap
    plot_heatmap(heatmap, denormalize(img[0]),  ax=ax)
    ax.set_title(idx2class[target])
fig.suptitle("model: {}".format(type(model).__name__))
plt.show()

## Monkey image

In [ ]:
img = np.array(Image.open("./monkeys.jpg"))
img = (img.transpose(2, 0, 1) / 255)
target = 382  # 382: squirrel monkey

# preprocess image
img  = Compose([Resize(image_size), ToTensor(),  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])(Image.open("./monkeys.jpg"))

model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target].mean()
heatmap = btln.heatmap(img[None].to(dev), model_loss_closure) 
ax = plot_heatmap(heatmap, denormalize(img))
_ = ax.set_title(idx2class[target])
plt.show()